In [ ]:
# general imports

import numpy as np
import scipy 
import pandas as pd
import os

In [ ]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
x_digits , y_digits= load_digits(return_X_y=True)
x_train, x_test, y_train, y_test = train_test_split(x_digits, y_digits, test_size=0.2, random_state=42)
x_train.shape
from sklearn.linear_model import LogisticRegression
log = LogisticRegression(multi_class='ovr', solver='lbfgs', max_iter=5000)
log.fit(x_train, y_train)
log_score = log.score(x_test,y_test)
log_score
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans

pipeline = Pipeline([
    ('cluster', KMeans(n_clusters=71)),
    ('log', LogisticRegression(solver='lbfgs', max_iter=5000))
])
pipeline.fit(x_train, y_train)
log_cluster_score = pipeline.score(x_test, y_test)
log_cluster_score
kmeans = KMeans(n_clusters=70, n_init='auto', random_state=42)
X_digits_dist = kmeans.fit_transform(x_train)
representative_digit_idx = np.argmin(X_digits_dist, axis=0)
X_representative_digits = x_train[representative_digit_idx]
import matplotlib.pyplot as plt
plt.figure(figsize=(9, 5))
for index, X_representative_digit in enumerate(X_representative_digits):
    plt.subplot(70 // 10, 10, index + 1)
    plt.imshow(X_representative_digit.reshape(8, 8), cmap="binary", interpolation="bilinear")
    plt.axis('off')

plt.show()
y_train[representative_digit_idx]
y_representative_digits = np.array([2, 9, 4, 6, 0, 7, 7, 5, 5, 4, 1, 1, 2, 3, 7, 3, 1, 4, 0, 8, 0, 6,
       9, 3, 8, 6, 1, 2, 5, 3, 8, 8, 5, 1, 2, 1, 5, 9, 1, 8, 9, 3, 4, 7,
       2, 8, 4, 5, 7, 6, 9, 9, 6, 3, 6, 7, 4, 5, 2, 9, 7, 2, 4, 3, 2, 1,
       5, 0, 7, 3])
log_reg = LogisticRegression(solver="lbfgs", max_iter=5000, random_state=42)
log_reg.fit(X_representative_digits, y_representative_digits)
log_reg.score(x_test, y_test)
y_train_propagated = np.empty(len(x_train), dtype=np.int32)
for i in range(70):
    y_train_propagated[kmeans.labels_==i] = y_representative_digits[i]
log_reg = LogisticRegression(multi_class="ovr", solver="lbfgs", max_iter=5000, random_state=42)
log_reg.fit(x_train, y_train_propagated)
log_reg.score(x_test, y_test)
# a little bit optimized after propagation but better than nothing due to the small number of labeled data
percentile_closest = 75

X_cluster_dist = X_digits_dist[np.arange(len(x_train)), kmeans.labels_]
for i in range(70):
    in_cluster = (kmeans.labels_ == i)
    cluster_dist = X_cluster_dist[in_cluster]
    cutoff_distance = np.percentile(cluster_dist, percentile_closest)
    above_cutoff = (X_cluster_dist > cutoff_distance)
    X_cluster_dist[in_cluster & above_cutoff] = -1
partially_propagated = (X_cluster_dist != -1)
X_train_partially_propagated = x_train[partially_propagated]
y_train_partially_propagated = y_train_propagated[partially_propagated]
log_reg = LogisticRegression(multi_class="ovr", solver="lbfgs", max_iter=5000, random_state=42)
log_reg.fit(X_train_partially_propagated, y_train_partially_propagated)
log_reg.score(x_test, y_test)
# it seems like the full propagated data is better
